# Assign NPIs!

In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from sodapy import Socrata

Get the 2016 open payments dataset

In [2]:
%%bigquery df
SELECT 
    Covered_Recipient_Type, 
    Teaching_Hospital_Name, 
    Physician_First_name, 
    Physician_Last_Name,
    Recipient_Primary_Business_Street_Address_Line1, 
    Recipient_City
FROM `carbide-server-294318.op2016.OP2016`

In [ ]:
%%bigquery npis
SELECT

FROM `carbide-server-294318.op2016.NPIs`

Modify the columns slightly

In [26]:
df.head()

,covered_recipient_type,teaching_hospital_name,physician_first_name,physician_last_name,recipient_primary_business_street_address_line1,recipient_city,npi
0,Covered Recipient Physician,None,SAUNDERS,HUPP,601 PROVIDENCE PARK DR E,MOBILE,NaN
1,Covered Recipient Physician,None,GARLAN,LO,1850 S AZUSA AVE,HACIENDA HEIGHTS,NaN
2,Covered Recipient Physician,None,GENE,KIM,6400 FANNIN ST,HOUSTON,NaN
3,Covered Recipient Physician,None,RAJIV,LUTHRA,2440 M ST NW,WASHINGTON,NaN
4,Covered Recipient Physician,None,STEVEN,GOLDENBERG,48 HARRISON STREET,JOHNSON CITY,NaN


In [5]:
# NPI matching
def npi_match(df):
    # for loop to go through every row in the OP dataframe
    # Source 3
    for index, row in df.iterrows():

        # Is the provider individual or enterprise?
        if row.covered_recipient_type == 'Covered Recipient Teaching Hospital':
            # Source 2
            results = npi.search(search_params={'organization_name': row.teaching_hospital_name,
                                                'address_purpose': 'PRIMARY',
                                                'city': row.recipient_city})                       

        if row.covered_recipient_type == 'Covered Recipient Physician':
            # Source 2
            results = npi.search(search_params={'first_name': row.physician_first_name,
                                                'last_name': row.physician_last_name,
                                                'address_purpose': 'PRIMARY',
                                                'city': row.recipient_city})

        # Iterate through the list of matches to find the exact match
        matches = 0
        for j in range(results['result_count']):
            if results['results'][j]['addresses'][0]['address_1'] == row.recipient_primary_business_street_address_line1:
                # We have a match!
                matches += 1
            
            # End loop early if more than one match was found
            if matches >= 1:
                break

        # Check to see if we only found one match
        #if matches == 1:
            #row.npi = results['results'][0]['number']
            
    return df

In [8]:
df.iloc[0].covered_recipient_type

'Covered Recipient Physician'

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client()

op_query = """
    SELECT 
        Covered_Recipient_Type, 
        Teaching_Hospital_Name, 
        Physician_First_name, 
        Physician_Last_Name,
        Recipient_Primary_Business_Street_Address_Line1, 
        Recipient_City
    FROM `carbide-server-294318.op2016.OP2016`
"""
query_job = client.query(op_query)  # Make an API request.

npi_query = """
    SELECT 
        NPI,
        Provider_Organization_Name__Legal_Business_Name_,
        Provider_Last_Name__Legal_Name_,
        Provider_First_Name,
        Provider_First_Line_Business_Practice_Location_Address,
        Provider_Business_Practice_Location_Address_City_Name
    FROM `carbide-server-294318.op2016.OP2016`
"""

print("The query data:")
for row in query_job:
    # Row values can be accessed by field name or index.
    if row.Covered_Recipient_Type == 'Covered Recipient Teaching Hospital':
        # Search through npi database for NPI bassed off values in row
    elif row.Covered_Recipient_Type == 'Covered Recipient Physician':
        # Search through npi database for NPI bassed off values in row
    

In [8]:
df.isnull().sum()

covered_recipient_type                                    0
teaching_hospital_name                             11655341
physician_first_name                                  41825
physician_last_name                                   41825
recipient_primary_business_street_address_line1           0
recipient_city                                            0
recipient_state                                         598
recipient_state_1                                       598
npi                                                11697166
dtype: int64

Sources
1. https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56014
2. https://npyi.readthedocs.io/en/latest/
3. https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas